In [106]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional, Reshape, Flatten
from tensorflow.keras.optimizers import Adam
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences


In [107]:
stories = [
""""The Forgotten Map": In a dusty old library, a young researcher discovers a mysterious map hidden within the pages of an ancient book. The map seems to lead to a long-lost kingdom filled with treasures and secrets. As the researcher delves deeper into deciphering the map's clues, they uncover unexpected challenges and adversaries determined to reach the treasure first.""",

""""Whispers in the Woods": In a secluded forest, strange whispers are heard by anyone who dares to enter. A curious teenager, known for their bravery, decides to investigate. As they venture deeper into the woods, they encounter enigmatic creatures and learn that the whispers hold the key to a forgotten prophecy that could change the fate of their world.""",

""""The Last Beacon": In a world plunged into darkness after an ancient catastrophe, a young engineer discovers an ancient lighthouse said to hold the power to restore light to the world. With the help of a group of unlikely allies, the engineer embarks on a perilous journey across treacherous lands, facing formidable challenges and adversaries who seek to keep the world in eternal darkness.""",

""""Echoes of Time": A gifted physicist invents a time-traveling device capable of sending messages to the past. However, the messages sent cause unforeseen ripples through time, altering events in unexpected ways. As the fabric of reality begins to unravel, the physicist races against time to correct the changes before irreparable damage occurs.""",

""""The Dream Catcher": In a town plagued by a series of haunting nightmares, a young artist discovers they have the ability to enter dreams and alter their course. Tasked with unraveling the mystery behind these nightmares, they navigate a surreal dream world, facing manifestations of people's deepest fears and secrets, all while trying to prevent a looming catastrophe that threatens to merge the dream realm with reality.""",
    
    # Add more stories here
]

In [108]:
# this checks the longest story with most number of words
max_len = max(len(story.split()) for story in stories)



In [109]:
vocab_size = 1000  # Choose an appropriate vocabulary size
embedding_dim = 100  # Embedding dimension

In [110]:
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(stories)


In [111]:
sequences = tokenizer.texts_to_sequences(stories)
sequences

[[1,
  22,
  11,
  5,
  2,
  42,
  43,
  44,
  2,
  12,
  23,
  13,
  2,
  45,
  11,
  46,
  47,
  1,
  48,
  4,
  14,
  15,
  49,
  1,
  11,
  50,
  3,
  51,
  3,
  2,
  52,
  53,
  54,
  55,
  9,
  56,
  6,
  24,
  16,
  1,
  23,
  57,
  25,
  17,
  58,
  1,
  59,
  60,
  7,
  61,
  26,
  27,
  6,
  28,
  62,
  3,
  63,
  1,
  64,
  65],
 [18,
  5,
  1,
  29,
  5,
  2,
  66,
  67,
  68,
  18,
  69,
  70,
  30,
  71,
  31,
  72,
  3,
  32,
  2,
  73,
  74,
  75,
  76,
  19,
  77,
  78,
  3,
  79,
  16,
  7,
  80,
  25,
  17,
  1,
  29,
  7,
  81,
  82,
  83,
  6,
  84,
  20,
  1,
  18,
  33,
  1,
  85,
  3,
  2,
  22,
  86,
  20,
  87,
  88,
  1,
  89,
  4,
  19,
  8],
 [1,
  90,
  91,
  5,
  2,
  8,
  92,
  17,
  34,
  93,
  14,
  15,
  35,
  2,
  12,
  36,
  13,
  14,
  15,
  94,
  95,
  3,
  33,
  1,
  96,
  3,
  97,
  98,
  3,
  1,
  8,
  9,
  1,
  99,
  4,
  2,
  100,
  4,
  101,
  102,
  1,
  36,
  103,
  104,
  2,
  105,
  106,
  107,
  108,
  109,
  37,
  110,
  27,
  6,
  28,

In [112]:
# [
#     [1, 2, 3, 4, 1, 5],
#     [1, 6, 7, 8, 9],
#     [10, 11, 12, 4, 1, 13]
# ]
## converts this code to this using
# padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')
# [
#     [1, 2, 3, 4, 1, 5],    # Unchanged - already at max_len
#     [1, 6, 7, 8, 9, 0],   # Padded with zeros at the end
#     [10, 11, 12, 4, 1, 13]  # Unchanged - already at max_len
# ]
# # 
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

In [113]:
# Building the generator model
generator = Sequential()
generator.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
generator.add(Bidirectional(LSTM(128)))
# activation='softmax' is used because the generator's output should represent 
# probabilities for each element in vocab_size in the vocabulary.
generator.add(Dense(vocab_size, activation='softmax'))

# Building the discriminator model
discriminator = Sequential()
discriminator.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
discriminator.add(Bidirectional(LSTM(128)))
discriminator.add(Dense(1, activation='sigmoid'))

discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

# Combining generator and discriminator as part of GAN
discriminator.trainable = False
gan_input = generator.input
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

In [121]:
# Training the GAN
epochs = 100 # Choose an appropriate number of epochs
batch_size = 32  # Define batch size
for epoch in range(epochs):
    # Generating noise data for the generator
    noise = np.random.randint(0, vocab_size, size=(batch_size, max_len))
    
    # Generating stories using the generator model
    generated_stories = generator.predict(noise)
    
    # Padding the generated stories to match the sequence length of real stories
    generated_stories_padded = pad_sequences(generated_stories, maxlen=max_len, padding='post')
    
    # Selecting real stories randomly from the dataset
    real_stories = padded_sequences[np.random.randint(0, len(padded_sequences), size=batch_size)]
    generated_stories
    # Concatenating real and generated stories
    x_combined = np.concatenate([real_stories, generated_stories_padded])
    
    # Creating labels for the combined real and generated stories
    y_combined = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    
    # Training the discriminator on the combined real and generated stories
    d_loss = discriminator.train_on_batch(x_combined, y_combined)
    
    # Generating new noise data for the generator
    noise = np.random.randint(0, vocab_size, size=(batch_size, max_len))
    
    # Creating labels for the generated data to mislead the discriminator
    y_mislabeled = np.ones((batch_size, 1))
    
    # Training the GAN (generator) to mislead the discriminator
    g_loss = gan.train_on_batch(noise, y_mislabeled)
    
    # Printing and tracking loss for observation
    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, Generator Loss: {g_loss}, Discriminator Loss: {d_loss}")
    
    





1/1 [==============================] - 0s 46ms/step


ValueError: in user code:

    File "c:\Users\baral\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\baral\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\baral\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\baral\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\baral\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\baral\App

In [ ]:
real_stories.shape, generated_stories.shape

((32, 67), (32, 67, 100))

In [ ]:
g_loss = gan.train_on_batch(noise, y_mislabeled)

g_loss

ValueError: in user code:

    File "c:\Users\baral\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\baral\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\baral\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\baral\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\baral\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\baral\App

In [ ]:
x_combined, y_combined.shape

(array([[114,   4,  10, ...,   0,   0,   0],
        [  1,  22,  11, ...,   0,   0,   0],
        [114,   4,  10, ...,   0,   0,   0],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]]),
 (64, 1))

In [ ]:
hehe = np.random.randint(0, len(padded_sequences), size=batch_size)
hehe
padded_sequences[hehe]

array([[ 18,   5,   1, ...,   0,   0,   0],
       [114,   4,  10, ...,   0,   0,   0],
       [114,   4,  10, ...,   0,   0,   0],
       ...,
       [114,   4,  10, ...,   0,   0,   0],
       [  1,  21, 142, ..., 171,   9,  40],
       [114,   4,  10, ...,   0,   0,   0]])

In [ ]:
# x_combined = np.concatenate([real_stories, generated_stories])
generated_stories.shape, generated_stories_padded.shape
generated_stories_padded, generated_stories
    # y_combined = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[9.98720134e-05, 1.00048579e-04, 9.99957701e-05, ...,
         9.99809563e-05, 9.98402975e-05, 1.00122576e-04],
        [1.00180543e-04, 1.00268917e-04, 9.99549738e-05, ...,
         1.00082485e-04, 1.00082834e-04, 1.00275967e-04],
        [1.00052443e-04, 1.00122641e-04, 1.00267549e-04, ...,
         1.00181518e-04, 1.00085963e-04, 1.00214784e-04],
        ...,
        [1.00170517e-04, 1.00021512e-04, 1.00016783e-04, ...,
         9.97081588e-05, 1.00030004e-04, 1.00188619e-04],
        [9.99131589e-05, 9.99968252e-05, 1.00044766e-04, ...,
         1.00066762e-04, 1.00083678e-04, 9.99090626e-05],
        [1.00117308e-04, 1.00013240e-04, 9.99811964e-05, ...,
         9.99144031e-05, 9.98461110e-05, 1.00163124e-04]], dtype=float32))